In [111]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim'

In [124]:
import requests
import json
import pandas as pd

url= "https://crashviewer.nhtsa.dot.gov/CrashAPI"
#/crashes/GetCrashesByLocation?fromCaseYear=2014&toCaseYear=2015&state=1&county=1&format=json

fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
qurl = f"{url}/crashes/GetCrashesByLocation?fromCaseYear={fromCaseYear}&toCaseYear={toCaseYear}&state={state}&county=73&format=json"

cali=requests.get(qurl).json()
data = (json.dumps(cali,indent=4,sort_keys=True))

CITYNAME = []
COUNTY =  []
COUNTYNAME = []
CaseYear = []
FATALS = []
LATITUDE = []
LONGITUD = []
STATE = []
STATENAME = []
ST_CASE = []
TOTALVEHICLES = []
TWAY_ID = []
TWAY_ID2 = []
VE_FORMS = []
# Loop through the list of cities and perform a request for data on each
for COUNTYNAME in COUNTYNAME:
    response = requests.get(qurl).json()
    CITYNAME.append(response[0]['CITYNAME'])


In [144]:
df = pd.DataFrame(cali['Results'][0])
# for i, r in df.iterrows():
#     print(i)
#     print(r["CaseYear"])
#     year = r["CaseYear"]
df[df['ST_CASE'] == '63474']

,CITY,CITYNAME,COUNTY,COUNTYNAME,CaseYear,FATALS,LATITUDE,LONGITUD,STATE,STATENAME,ST_CASE,TOTALVEHICLES,TWAY_ID,TWAY_ID2,VE_FORMS
1890,710,CHULA VISTA,73,SAN DIEGO (73),2016,1,32.60915556,-117.019969440,6,California,63474,2,OLYMPIC PKWY,,2


In [127]:
fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"

qurl = f"{url}/crashes/GetCaseDetails?stateCase=63474&caseYear=2016&state=6&format=json"
data = requests.get(qurl).json()
case = data['Results'][0][0]['CrashResultSet']
case['Vehicles'][0]['Persons']

[{'AGE': '40',
  'AGENAME': '40 Years',
  'AIR_BAG': '2',
  'AIR_BAGNAME': 'Deployed- Side (door, seatback)',
  'ALC_DET': '9',
  'ALC_DETNAME': 'Not Reported',
  'ALC_RES': '996',
  'ALC_RESNAME': 'Test Not Given',
  'ALC_STATUS': '0',
  'ALC_STATUSNAME': 'Test Not Given',
  'ATST_TYP': '0',
  'ATST_TYPNAME': 'Test Not Given',
  'BODY_TYP': '14',
  'BODY_TYPNAME': 'Compact Utility (Utility Vehicle Categories "Small" and "Midsize")',
  'CERT_NO': None,
  'COUNTY': '73',
  'COUNTYNAME': 'SAN DIEGO (73)',
  'CaseYear': '2016',
  'DAY': '25',
  'DEATH_DA': '88',
  'DEATH_DANAME': 'Not Applicable (Non-Fatal)',
  'DEATH_HR': '88',
  'DEATH_HRNAME': 'Not Applicable (Non-fatal)',
  'DEATH_MN': '88',
  'DEATH_MNNAME': 'Not Applicable (Non-fatal)',
  'DEATH_MO': '88',
  'DEATH_MONAME': 'Not Applicable (Non-Fatal)',
  'DEATH_TM': '8888',
  'DEATH_YR': '8888',
  'DEATH_YRNAME': 'Not Applicable (Non-fatal)',
  'DOA': '0',
  'DOANAME': 'Not Applicable',
  'DRINKING': '0',
  'DRINKINGNAME': 'No (Alc

In [136]:
def extract_people(v):
    for p in v['Persons']:
        yield {
            'Speed Limite Exceeded': v['SPEEDRELNAME'],
            'Speed limite': v['TRAV_SP'],
            'Vin Number': v['VINNAME'],
            'Traveled Speed Veh': v['VSPD_LIM'],
            'Make': v['MAKENAME'],
            'Make/Model': v['MAK_MODNAME'],
            'Model': v['MODELNAME'],
            
            "Age": p['AGE'],
            "Age Name": p['AGENAME'],
            "County": p['COUNTYNAME'],
            "Death": p['DEATH_DANAME'],
            "DOA Name": p['DOANAME'],
            "Injury": p['INJ_SEVNAME'],
            "Race": p['RACENAME'],
            "Road_Type": p["ROAD_FNCNAME"],
            "Sex": p["SEXNAME"],
            "Make": p["MAKENAME"],
        }


        
def get_people(case):

    hour = case['HOUR']
    minute = case['MINUTE']
    time = f"{hour}:{minute}"
        
    accident_info = {
        'Case Number': case['ST_CASE'],
        "Description of Veh Coll": case['CF2NAME'], 
        "Day of Week": case['DAY_WEEKNAME'],
        "Drunk Driver": case['DRUNK_DR'],
        "Year": case['CaseYear'],
        "Month": case['MonthName'],
        "Hour": hour,
        "Time of Accident": time,
    }

    vehicles = case['Vehicles']
    
    people = [{**accident_info, **p} for v in vehicles for p in extract_people(v)]
    
    return pd.DataFrame(people)


def get_events(case):
    c_events = [{
        'Case Number': case['ST_CASE'],
# In a traffic accident AOI is Area of Impact. The spot the two cars collided is measured 
# to a fixed object, usually the curb, so it can be reconstructed later.
        'Area of Impact': e['AOI1NAME'],
# standard of evidence
# https://safety.fhwa.dot.gov/rsdp/cdip_rpti.aspx
        'Standard of Evenidence': e['SOENAME'],
        'Event Number': e['EVENTNUM'],
        'Vehicle 1': e['VNUMBER1'],
        'Vehicle 2': e['VNUMBER2'],
    } for e in case['CEvents']]
    
    return pd.DataFrame(c_events)




In [137]:
get_people(case)

,Age,Age Name,Case Number,County,DOA Name,Day of Week,Death,Death DANAME,Description of Veh Coll,Drunk Driver,...,Month,Race,Road_Type,Sex,Speed Limite Exceeded,Speed limite,Time of Accident,Traveled Speed Veh,Vin Number,Year
0,40,40 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable,Not Applicable (Non-Fatal),None,0,...,November,Not a Fatality (not Applicable),None,Male,No,998,11:51,50,2HKRM4H79EH7,2016
1,36,36 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable,Not Applicable (Non-Fatal),None,0,...,November,Not a Fatality (not Applicable),None,Female,No,998,11:51,50,2HKRM4H79EH7,2016
2,11,11 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable,Not Applicable (Non-Fatal),None,0,...,November,Not a Fatality (not Applicable),None,Female,No,998,11:51,50,2HKRM4H79EH7,2016
3,22,22 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable,26,None,0,...,November,Black,None,Male,"Yes, Too Fast for Conditions",77,11:51,50,WBAAM3340YKC,2016
4,19,19 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable,Not Applicable (Non-Fatal),None,0,...,November,Not a Fatality (not Applicable),None,Male,"Yes, Too Fast for Conditions",77,11:51,50,WBAAM3340YKC,2016


In [130]:
get_events(case)

,Area of Impact,Case Number,Event Number,Standard of Evenidence,Vehicle 1,Vehicle 2
0,Right,63474,1,Motor Vehicle In-Transport,1,2
1,Non-Harmful Event,63474,2,Ran Off Roadway - Right,1,5555
2,Not Reported,63474,3,Curb,1,9999
3,Not Reported,63474,4,Tree (Standing Only),1,9999
